<a href="https://colab.research.google.com/github/JonasVerbickas/test-jupyter/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
from bs4 import BeautifulSoup
import time
import re
import tqdm

In [2]:
s = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
          "Accept-Language":"en-US,en;q=0.9,lt;q=0.8"}
s.headers.update(headers)

r = s.get("http://hex.kosmanor.com/hex-bin/newdump/51233", headers=headers)


In [16]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

<html>
<head>
<title>OHex game 51233</title>
</head>
<body bgcolor="#E0E0E0">
<h3>OHex game number 51233</h3>

Origin: \completion G50842C.gam after move 22<br/>
V played vertically, H played horizontally.<br/>
And the winner was V.<br/>
<br/>
Played at Thu Jan 17 17:05:22 2002 UTC on a size-11 board.<br/>
Status <a href="../../hex/status.html">flags</a>: "HCcKWS"<br/>
References: <a href="50842">B50842</a><br/>
Click <a href="/hex-bin/hexy/OHex-51233.gam">here</a> for a Hexy game record
<pre>
Raw moves: <br/>
Std moves: CgFbCdCeDeDdGcFdGdFfFeDgBiDfHeEeGgHaIbHiFiJaIaHcHbGeHdHfGfFhGhFjEjEkDk<br/>
Standard moves follow:
         1.     C7    F2
         2.     C4    C5
         3.     D5    D4
         4.     G3    F4
         5.     G4    F6
         6.     F5    D7
         7.     B9    D6
         8.     H5    E5
         9.     G7    H1
        10.     I2    H9
        11.     F9    J1
        12.     I1    H3
        13.     H2    G5
        14.     H4    H6
        15.     G6    F8

In [17]:
soup.getText()

'\n\nOHex game 51233\n\n\nOHex game number 51233\n\nOrigin: \\completion G50842C.gam after move 22\nV played vertically, H played horizontally.\nAnd the winner was V.\n\nPlayed at Thu Jan 17 17:05:22 2002 UTC on a size-11 board.\nStatus flags: "HCcKWS"\nReferences: B50842\nClick here for a Hexy game record\n\nRaw moves: \nStd moves: CgFbCdCeDeDdGcFdGdFfFeDgBiDfHeEeGgHaIbHiFiJaIaHcHbGeHdHfGfFhGhFjEjEkDk\nStandard moves follow:\n         1.     C7    F2\n         2.     C4    C5\n         3.     D5    D4\n         4.     G3    F4\n         5.     G4    F6\n         6.     F5    D7\n         7.     B9    D6\n         8.     H5    E5\n         9.     G7    H1\n        10.     I2    H9\n        11.     F9    J1\n        12.     I1    H3\n        13.     H2    G5\n        14.     H4    H6\n        15.     G6    F8\n        16.     G8   F10\n        17.    E10   E11\n        18.    D11\n\n\n\n'

In [14]:
%%time
re.findall('size-11', soup.get_text())

CPU times: user 74 µs, sys: 7 µs, total: 81 µs
Wall time: 86.1 µs


['size-11']

In [15]:
%%time
'size-11' in soup.get_text()

CPU times: user 51 µs, sys: 5 µs, total: 56 µs
Wall time: 59.6 µs


True

In [42]:
pages = []

STARTING_I = 51237

for i in tqdm.tqdm(range(STARTING_I, STARTING_I+10)):
    url = f"http://hex.kosmanor.com/hex-bin/newdump/{i}"
    r = s.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    if 'size-11' in soup.get_text():
      matches = re.findall("\d+\.(\s+[A-K]\d{1,2}){1,2}", soup.get_text())
      print('matches', matches)
    time.sleep(2)

with open("output1.html", "w") as file:
    file.write(str(pages))

  0%|          | 0/10 [00:00<?, ?it/s]

matches ['    E8', '    G8', '   E10', '   F10', '   G11', '   I10', '    E9', '   B10', '   A11', '    D8', '   C11', '   C10', '   D11', '   H11', '   F11', '    J9', '    K8']


 10%|█         | 1/10 [00:02<00:23,  2.59s/it]

matches ['    F4', '    H3', '    E6', '    E3', '    C3', '    D9', '   B10', '    E1', '    F1', '    D1', '    C1', '    B2', '    F7', '    G3', '    H4', '    B3', '    B4', '    B1', '    B5', '    B6', '    B7', '    B8', '    K1', '    H6', '    F6', '    G6', '    H7', '    A9', '    E2', '    F5', '    I5', '    E4', '    J4', '    J3', '    J2']


 20%|██        | 2/10 [00:05<00:20,  2.61s/it]

matches ['    C9', '   G10', '    E9', '   E10', '   F11', '    E7', '    F5', '    G5', '    H3', '    G4', '    J1', '    I1', '    G7', '    H1', '    F3', '    G1', '     F1']


 30%|███       | 3/10 [00:08<00:20,  2.92s/it]


KeyboardInterrupt: ignored

In [ ]:
X = []
Y = []
for i, page in enumerate(pages):
    for listing in page:
        x = listing.select_one('.listing-key-specs').text
        # Only the 7 features are consistent
        # Further positions have unpredictable features in them
        # e.g. ['2004 (04 reg)', 'Convertible', '59,100 miles', '3.0L', '231BHP', 'Manual', 'Petrol', '5 owners', 'Full service history']
        x_list = x.strip().split('\n')[:7]
        X.append(x_list)
        y = listing.select_one('.product-card-pricing__price').span.text
        Y.append(y)
print(len(X))
print(len(Y))

154
154


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(X, columns=["Year", "Type", "Miles", "Liters", "BHP", "Transmission", "Fuel"])

In [ ]:
df["Price"] = Y

In [ ]:
df.to_csv("df.csv")
df

,Year,Type,Miles,Liters,BHP,Transmission,Fuel,Price
0,2007 (57 reg),Saloon,"152,000 miles",2.0L,150BHP,Manual,Petrol,£990
1,2011 (60 reg),Hatchback,"115,000 miles",2.0L,116BHP,Manual,Diesel,"£1,595"
2,2006 (56 reg),Coupe,"116,000 miles",3.0L,306BHP,Manual,Petrol,"£3,200"
3,2006 (56 reg),Saloon,"74,000 miles",2.0L,129BHP,Manual,Petrol,"£2,285"
4,2007 (08 reg),Hatchback,"150,396 miles",2.0L,143BHP,Manual,Diesel,"£1,595"
...,...,...,...,...,...,...,...,...
149,2007 (07 reg),Coupe,"77,000 miles",2.0L,177BHP,Automatic,Diesel,"£4,500"
150,2002 (02 reg),Saloon,"118,830 miles",3.0L,228BHP,Automatic,Petrol,"£3,295"
151,2009 (58 reg),Coupe,"106,000 miles",3.0L,197BHP,Automatic,Diesel,"£5,895"
152,2009 (09 reg),Saloon,"83,000 miles",2.0L,177PS,Manual,Diesel,"£5,590"
